# Explore Data
Notebook for initial pulling and performing EDA on NFL Combine data to compare with overall NFL success.

In [5]:
# !pip install --upgrade pip
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !pip install statsmodels
# !pip install beautifulsoup4

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import requests
from bs4 import BeautifulSoup
import re
import datetime as dt
import time

%matplotlib inline

In [3]:
url = r'https://nflcombineresults.com/nflcombinedata.php?year=all&pos=QB&college='

# Get page content
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
def get_combine(soup=None):
    """Function to grab all Quarterback combine data. Returns DataFrame of each QB that
       participated in the NFL Combine from 1987-present. The data returned includes:
        - year: Year of participation
        - name: Player Name
        - college: College attended
        - pos: Player Position (QB in this case)
        - height: Player Height (inches)
        - weight: Player Weight (lbs)
        - wonderlic: Wonderlic Cognitive Ability Test Score (0-50 overall, 20 avg intelligence)
        - 40_yard: 40 Yard Dash Time (s)
        - bench_press: Number of 225 lb Repititions
        - vert_leap: Standing Vertical Jump Height (inches)
        - broad_jump: Standing Long Jump Distance (inches)
        - shuttle: 20 Yard Shuttle Time (s)
        - 3cone: 3-Cone Drill Time (s)"""
    
    # Check if soup is not provided
    if soup is None:
        durl = r'https://nflcombineresults.com/nflcombinedata.php?year=all&pos=QB&college='
        page = requests.get(durl)
        soup = BeautifulSoup(page.content, 'html.parser')
    
    # Find table on the webpage and all trs
    table = soup.find('table', attrs={'class': 'sortable'})
    trs = table.find_all('tr')
    
    # Parse how to go thru trs and tds
    rows = []
    for i, tr in enumerate(trs):
        tds = tr.find_all('td')

        # First tr is the header row
        if i==0:
            header = [td.text.lower()
                             .replace(' (in)','')
                             .replace(' (lbs)','')
                             .replace(' ','_')
                      for td in tds]
            continue
        # Break loop if tr is looking at footer
        elif i==len(trs)-1:
            break
            
        # Read each tr as a row
        rows.append([td.text for td in tds if i!=0])

    return pd.DataFrame(rows, columns=header)

In [27]:
pd.set_option('display.min_rows',100)
combine = get_combine(soup)
combine.head(15)

,year,name,college,pos,height,weight,wonderlic,40_yard,bench_press,vert_leap,broad_jump,shuttle,3cone
0,2021,Ian Book,Notre Dame,QB,72.00,211,,4.65,,32.5,115,4.20,7.00
1,2021,Shane Buechele,Southern Methodist (TX),QB,72.25,210,,4.96,,30.0,114,4.60,7.34
2,2021,K.J. Costello,Mississippi State,QB,76.50,225,,5.11,,27.5,103,4.75,
3,2021,Sam Ehlinger,Texas,QB,73.13,220,,4.84,,36.5,114,4.44,7.15
4,2021,Justin Fields,Ohio State,QB,74.75,227,,4.46,,,119,,
5,2021,Feleipe Franks,Arkansas,QB,78.63,234,,4.61,,32.5,117,4.22,7.16
6,2021,Trey Lance,North Dakota State,QB,75.88,224,,,,,131,,
7,2021,Trevor Lawrence,Clemson,QB,77.63,213,,,,38.5,119,,
8,2021,Kellen Mond,Texas A&M,QB,74.63,211,,4.62,,,122,,
9,2021,Jamie Newman,Wake Forest,QB,74.88,234,,,,,114,,


In [201]:
# Grab NFL draft data from 1987-present, clean and then merge into combine data
# Note 1987-1991 uses different formatting than the rest of the years
ele_num={ '1987': list(range(2,14)),
          '1988': list(range(2,14)),
          '1989': list(range(2,14)),
          '1990': list(range(2,14)),
          '1991': list(range(2,14)),
          '1992': 2,
          '1993': 4,
          '1994': 4,
          '1995': 4,
          '1996': 4,
          '1997': 4,
          '1998': 4,
          '1999': 5,
          '2000': 4,
          '2001': 4,
          '2002': 4,
          '2003': 5,
          '2004': 5,
          '2005': 5,
          '2006': 5,
          '2007': 5,
          '2008': 5,
          '2009': 5,
          '2010': 6,
          '2011': 5,
          '2012': 5,
          '2013': 5,
          '2014': 5,
          '2015': 4,
          '2016': 4,
          '2017': 4,
          '2018': 4,
          '2019': 4,
          '2020': 4,
          '2021': 4,
         }
# Create a map to rename columns
map_ = {'Pick #': 'pick',
        'Pick No': 'pick',
        'NFL team': 'team',
        'NFL Team': 'team',
        'Player': 'name',
        'Position': 'pos',
        'Pos': 'pos',
        'College': 'college',
        'Conf': 'college_conf',
        'Rnd': 'round',
        'Notes': 'notes'
       }

years = list(range(1987,2022))
# years = list(range(2018,2022))
# years = list(range(1987,1993))
drafts = []
for year in years:
    ele = ele_num[str(year)]  # Each year has a different list element number used
    url = f'https://en.wikipedia.org/wiki/{year}_NFL_Draft'
    try:
        if int(year)<=1991:
            df = pd.read_html(url)
            rounds = list(range(1,13))
            for rnd in rounds:
                df_rnd = df[int(rnd)+1]
                df_rnd['Rnd'] = rnd
                if rnd==1:
                    draft = pd.DataFrame(df_rnd)
                else:
                    draft = pd.concat([draft, df_rnd], axis=0)
            draft = draft[draft['Position']=='Quarterback']
            draft = draft.rename(columns=map_)
        else:
            draft = pd.read_html(url)[ele]
            draft = draft[draft['Pos.']=='QB']
            draft = draft.drop('Unnamed: 0', axis=1)
            draft = draft.rename(columns=lambda x: re.sub(r'\.','',x))
            draft = draft.rename(columns=map_)
    except:
        print(f'WARNING: {year} failed to read')
    else:
        draft['year'] = year  # add year column to each draft
        drafts.append(draft)
        print(f'SUCCESS: {year} read into drafts list')
        time.sleep(2)

qb_drafts = pd.concat(drafts).sort_values(['year','round','pick']).reset_index()
qb_drafts = qb_drafts[['year','round','pick','name','college','college_conf','team','notes']]

SUCCESS: 1987 read into drafts list
SUCCESS: 1988 read into drafts list
SUCCESS: 1989 read into drafts list
SUCCESS: 1990 read into drafts list
SUCCESS: 1991 read into drafts list
SUCCESS: 1992 read into drafts list
SUCCESS: 1993 read into drafts list
SUCCESS: 1994 read into drafts list
SUCCESS: 1995 read into drafts list
SUCCESS: 1996 read into drafts list
SUCCESS: 1997 read into drafts list
SUCCESS: 1998 read into drafts list
SUCCESS: 1999 read into drafts list
SUCCESS: 2000 read into drafts list
SUCCESS: 2001 read into drafts list
SUCCESS: 2002 read into drafts list
SUCCESS: 2003 read into drafts list
SUCCESS: 2004 read into drafts list
SUCCESS: 2005 read into drafts list
SUCCESS: 2006 read into drafts list
SUCCESS: 2007 read into drafts list
SUCCESS: 2008 read into drafts list
SUCCESS: 2009 read into drafts list
SUCCESS: 2010 read into drafts list
SUCCESS: 2011 read into drafts list
SUCCESS: 2012 read into drafts list
SUCCESS: 2013 read into drafts list
SUCCESS: 2014 read into draf

In [263]:
# Remove special characters and convert to numeric
qb_drafts['round'] = pd.to_numeric(qb_drafts['round'].copy().replace(regex=True, to_replace=r'[^0-9.\-]', value=r''))

# Set picks to numbers and for nans if not possible then drop those rows
qb_drafts['pick'] = pd.to_numeric(qb_drafts['pick'], errors='coerce', downcast='integer')
qb_drafts = qb_drafts.dropna(subset=['pick'])

# Clean name column
qb_drafts['name'] = qb_drafts['name'].str.split(r'\[|\(').str[0]
qb_drafts['name'] = qb_drafts['name'].str.replace('†','').str.rstrip()

# Use notes column to generate pick traded column and heisman column
qb_drafts['heisman'] = qb_drafts['notes'].str.lower().str.contains(r'heisman').fillna(False).astype('int')
qb_drafts['pick_traded'] = qb_drafts['notes'].str.lower().str.contains(r'from').fillna(False).astype('int')

# Clean team column
idx = qb_drafts.index[qb_drafts['name']=='Jeff George']
qb_drafts.loc[idx,'team'] = qb_drafts.loc[idx,'team'].values[0].split('(')[0].rstrip()
qb_drafts.loc[idx,'pick_traded'] = 1

# Drop notes column
qb_drafts = qb_drafts.drop('notes', axis=1)

# Clean up colleges
college_replace = {r'Hawaiʻi': 'Hawaii',
                   r'Wis\.': 'WI',
                   'Louisiana State': 'LSU',
                   'N.C. State': 'NC State',
                   'North Carolina State': 'NC State',
                   'Southern Mississippi': 'Southern Miss',
                   'Texas-El Paso': 'UTEP',
                   'California-Davis': 'UC Davis'}
qb_drafts['college'] = qb_drafts['college'].college.replace(college_replace, regex=True)

In [25]:
# Grab data from NFL Next Gen Stats Website
url = r'https://nextgenstats.nfl.com/stats/passing#yards'
df = pd.read_html(url)

ValueError: No tables found

In [ ]:
# Show Andrew statsmodels package